In [1]:
!pip install boto3 pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 210.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 161.4 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 304.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 7.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 7.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 1.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 449.8 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 638.1 kB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import boto3
from botocore.client import Config

# Настройки MinIO (замените на свои)
MINIO_ENDPOINT = "http://minio:9000"  # имя сервиса из docker-compose
ACCESS_KEY = "minio"
SECRET_KEY = "minio-password"

# Создаём клиент
s3 = boto3.client(
    's3',
    endpoint_url=MINIO_ENDPOINT,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    config=Config(signature_version='s3v4'),
)

In [3]:
bucket_name = "my-bucket"

if bucket_name not in [b['Name'] for b in s3.list_buckets()['Buckets']]:
    s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket {bucket_name} created!")
else:
    print(f"Bucket {bucket_name} already exists")

Bucket my-bucket created!


In [4]:
import pandas as pd

# Пример DataFrame
df = pd.DataFrame({
    "name": ["Alice", "Bob", "Charlie"],
    "age": [25, 30, 35],
    "city": ["NY", "LA", "Chicago"]
})

# Сохраняем локально в CSV
csv_file = "example.csv"
df.to_csv(csv_file, index=False)

In [5]:
s3.upload_file(csv_file, bucket_name, csv_file)
print(f"{csv_file} uploaded to {bucket_name}")

example.csv uploaded to my-bucket


In [6]:
from io import BytesIO

obj = s3.get_object(Bucket=bucket_name, Key=csv_file)
df_from_minio = pd.read_csv(BytesIO(obj['Body'].read()))
print(df_from_minio)

      name  age     city
0    Alice   25       NY
1      Bob   30       LA
2  Charlie   35  Chicago
